In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from linear_transform import LinearTransform
from pearsonr import PearsonR
from utils import *
from tqdm import tqdm

In [13]:
trained_probe = LinearTransform()
trained_probe.load_state_dict(torch.load('first_full_run.pth',map_location=torch.device('cpu')))
trained_probe.eval()

LinearTransform(
  (conv_layer): Conv1d(1536, 18, kernel_size=(1,), stride=(1,))
  (activation): Softplus(beta=1, threshold=20)
)

In [9]:
predictions_savepath = '/clusterfs/nilah/oberon/datasets/cs282a/inference/probe_first_full_run.h5'
f = h5py.File('/clusterfs/nilah/oberon/datasets/basenji/embeddings/embeddings.h5','r')
dset = f['embeddings']

In [18]:
with h5py.File(predictions_savepath,'w') as savefile:
    savefile.create_dataset(
        '128bp_bins',
        shape=(len(dset),896,18),
        chunks=(1,896,18),
        compression='gzip',
        compression_opts=9,
    )
    for i in tqdm(range(len(dset))):
        inputs = torch.Tensor(f['embeddings'][i])
        predictions = trained_probe(inputs.transpose(0,1))
        savefile['128bp_bins'][i,:,:] = predictions.detach().transpose(0,1)

100%|██████████| 38171/38171 [29:08<00:00, 21.83it/s]  
